In [1]:
from copy import deepcopy
import torch

from yolov7_utils.torch_utils import is_parallel
from nas.supernet.yolo import YOLOModel
from nas.supernet.supernet_yolov7 import YOLOSuperNet

/home/heonsung/anaconda3/envs/lab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'TinyELAN' from 'nas.supernet.search_block' (/home/heonsung/Desktop/yolov7-nas-for-TANGO/nas/supernet/search_block.py)

In [ ]:
supernet = YOLOSuperNet(cfg='./yaml/yolov7_supernet.yml', ch=3)
net = YOLOModel(cfg='./yaml/yolov7.yaml', ch=3)

In [ ]:
# It needs models module in yolov7 to load weights when it was saved.
weights = 'yolov7.pt'
device = 'cpu'
ckpt = torch.load(weights, map_location=device)

In [ ]:
for superkey, yolokey in zip(supernet.state_dict(), ckpt['model'].state_dict()):
    print(f'{superkey} | {yolokey}')

In [ ]:
for key in supernet.state_dict():
    print(key)

In [ ]:
# if we use IDetect layer, it has 6 more state_dict than yolov7.pt
# because IDetect layer has 6 more parameters than Detect layer.
len(supernet.state_dict()), len(ckpt['model'].state_dict())

In [ ]:
def load_weights_from_yolo(supernet, yolo_dict):
        super_dict = supernet.state_dict()

        for superkey, yolokey in zip(super_dict, yolo_dict):
            super_dict[superkey] = yolo_dict[yolokey] # yolo state와 supernet state 1:1 매칭

        supernet.load_state_dict(super_dict) # load weight
        print('All keys matched successfully')
        
        return supernet

In [ ]:
supernet_weighted = load_weights_from_yolo(supernet, ckpt['model'].state_dict())

In [ ]:
ckpt

In [ ]:
ckpt = {
        'model': deepcopy(supernet_weighted.module if is_parallel(supernet_weighted) else supernet_weighted).half(),
}

torch.save(ckpt, 'yolov7_supernet.pt')

In [ ]:
for m in supernet_weighted.model:
    print(m)
    break

In [ ]:
torch.save(torch.load('yolov7.pt')['model'].state_dict(), 'yolov7_statedict.pt') # save yolov7 statedict
yolo_weight = torch.load('yolov7_statedict.pt') # load yolov7 statedict

In [ ]:
supernet_pt = load_weights_from_yolo(supernet, yolo_weight)

In [ ]:
# check weight
x = supernet_weighted.state_dict()['model.4.layers.1.conv.weight'].cuda()
y = yolo_weight['model.5.conv.weight']

print('Supernet BBoneELAN Conv // shape :', x.shape)
print('weight :', x.flatten())
print('---')
print('YOLOv7 BBoneELAN Conv  // shape :', y.shape)
print('weight :', y.flatten())
print('---')
print('Is same?: ', (x == y).flatten())

In [ ]:
torch.save({'model': supernet_pt}, 'yolov7_supernet_pt.pt') # save supernet statedict